In [1]:
import time
import cv2
import pickle as pkl
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
from IPython import display as dp
from collections import deque
import matplotlib.animation as animation

In [2]:
def drawRectangleOnImage(img, bbox, track=0, line=[], colour=(0, 255, 0)):
    b, g, r = colour
    img = cv2.rectangle(img, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (int(b), int(g), int(r)), 3)
    img = cv2.putText(img, str(track), (bbox[0], bbox[1] - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (int(b), int(g), int(r)), 3)
    for i in range(1, len(line)):
        img = cv2.line(img, line[i - 1], line[i], (int(b), int(g), int(r)), 3)
    return img

In [3]:
def get_box_center(bbox):
    center = (int(bbox[0]) + ((int(bbox[2]) - int(bbox[0])) // 2)), (int(bbox[1]) + ((int(bbox[3]) - int(bbox[1])) // 2)) # X + Width / 2, Y + Height / 2
    return center

In [4]:
# Detection Pre-Processing 

# Remove overlaps in the same frame
def remove_overlaps(detections_pd, tolerance=0.9):
    row_to_remove = []
    for detection in detections_pd.get('detection'):
        length, _ =detections_pd.shape
        for i in range(length):
            IoU = detection.IoU(detections_pd.iloc[i]['detection'])
            if IoU > tolerance and IoU < 1:
                row_to_remove.append(i)
                
    row_to_remove = np.unique(np.array(row_to_remove))
    detections_pd = detections_pd.drop(index=row_to_remove)
        
    return detections_pd

In [5]:
# Detection to DataFrame
def get_detection_dataframe(detections):
    bboxes = []
    bsizes = []
    lines = deque(maxlen=32)
    bdetections = []
    tracks = list(range(0, len(detections)))
    colours = []
    for i in range(len(detections)):
        colours.append(tuple(np.random.choice(range(256), size=3).astype('int')))
        
    updated = [False]*len(detections)
    
    for detection in detections:
        bbox = np.array(detection.getBBox()).astype('int')
        bboxes.append(bbox)
        
        centers = []
        centers.append(get_box_center(bbox))
        lines.append(centers)

        bsize = int(detection.areaOfRec())
        bsizes.append(bsize)

        bdetections.append(detection)

    detec = {
        'track': tracks,
        'detection': bdetections,
        'bbox': bboxes,
        'size': bsizes,
        'line': lines,
        'colour': colours,
        'updated': updated
    }
    detections_pd = pd.DataFrame(detec)
    detections_pd = detections_pd.sort_values(by=['size'], ascending=False)
    detections_pd = detections_pd.reset_index(drop=True)
    
    return detections_pd

In [6]:
def get_new_track_id(array):
    for cursor in range(30):
        if array[cursor] != cursor:
            return cursor
    return N

In [7]:
def update_track(detections_pd, next_detections_pd, tolerance=0.5):
    detections_pd['updated'] = False
    detections_pd = detections_pd.reset_index(drop=True)
    for index, next_detection in next_detections_pd.iterrows():
        length, _ = detections_pd.shape
        
        # Find overlaps and update if found
        for i in range(length):
            IoU = next_detection['detection'].IoU(detections_pd.iloc[i]['detection'])
            if IoU > tolerance and detections_pd['updated'].iloc[i] != True:
                detections_pd.at[i,'detection'] = next_detection['detection']
                detections_pd.at[i,'bbox'] = next_detection['bbox']
                detections_pd.at[i,'size'] = next_detection['size']
                detections_pd.at[i,'line'].append(next_detection['line'][0])
                detections_pd.at[i,'updated'] = True
                next_detections_pd.at[index, 'updated'] = True
                break
    
    # Drop detections no longer exist
    detections_pd = detections_pd[detections_pd['updated'] == True]
                
    # Start tracking new detections
    if any(next_detections_pd['updated'] == False):
        new_pd = next_detections_pd[next_detections_pd['updated']==False]
        new_pd = new_pd.reset_index(drop=True)
        
        print('before',new_pd)
        
        #New Track Number for detection
        
#         max_tracking_id = detections_pd['track'].max()
    
#         for i in range(len(new_pd)):
#             new['track'] = max_tracking_id + 1 + index
        
        
        counter = 0
        for index, new in new_pd.iterrows():
            while counter in detections_pd['track']:
                counter = counter + 1
            new_pd.at[index, 'track'] = counter
            counter = counter + 1

#         track_list = detections_pd['track'].tolist()
#         track_list = track_list.sort()
#         for index, new in new_pd.iterrows():
#             new_track_id = get_new_track_id(track_list)
#             new_pd.at[index, 'track'] = new_track_id
#             track_list.append(new_track_id)

        detections_pd = pd.concat([detections_pd, new_pd])
    detections_pd = detections_pd.reset_index(drop=True)
    return detections_pd

In [8]:
# Load paths
# detection_path = 'detection_pkls/retinanet_101_detections.pkl'
detection_path = 'detection_pkls/maskRCNN_101_detections.pkl'
data_path = '../datasets/AICity_data/train/S03/c010/'

# Load detections
with open(detection_path , 'rb') as f:
    all_detections = pkl.load(f)

In [9]:
# Pre-process the first frame if needed
# detections_pd = remove_overlaps(detections_pd, 0.5)
# detections_pd

In [10]:
display = False
gif = False

if gif:
    fig, ax = plt.subplots()
    plt.axis('off')

ims = []
detection_history = []

# Get the First Frame
detections_pd = get_detection_dataframe(all_detections['0'])
detection_history.append(detections_pd)

# Load video
vidcap = cv2.VideoCapture(data_path + 'vdo.avi')
_, image = vidcap.read()
num_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))

# Set plot
if display:
    plt.ion() 
    plt.figure(figsize=(20, 12))
    plt.axis('off') 

# Iterate Frames
for frame in tqdm(range(1,num_frames//4)):
    for i in range(4):
        _, image = vidcap.read()
    frame = frame*4
    
    next_detections_pd = get_detection_dataframe(all_detections[str(frame)])
    detections_pd = update_track(detections_pd, next_detections_pd, tolerance=0.5)
    detection_history.append(detections_pd)
    
    for index, row in detections_pd.iterrows():
        image = drawRectangleOnImage(image, row['bbox'], track=row['track'], line=row['line'],colour=row['colour'])
        
    if display:
        plt.title('Frame:'+str(frame))
        plt.imshow(image)  
    
    if gif:
        im = ax.imshow(image, animated=True)
        ims.append([im])
        
    if display:
        dp.clear_output(wait=True)
        dp.display(plt.gcf())
        time.sleep(0.000001)
        plt.cla()

if gif:
    ani = animation.ArtistAnimation(fig, ims, interval=10, blit=True, repeat_delay=10000)
    ani.save('Tracking_with_IoU_overlap' + ".gif", writer=animation.PillowWriter(fps=24))
    
# Open a file and use dump()
with open('tracking_history/tracking_history.pkl', 'wb') as file:
    # A new file will be created
    pkl.dump(detection_history, file)

100%|██████████| 534/534 [00:15<00:00, 35.08it/s]


In [11]:
with open('tracking_history/tracking_history.pkl', 'rb') as file:
    # Call load method to deserialze
    tracking_hist = pkl.load(file)

In [12]:
tracking_hist[83]

,track,detection,bbox,size,line,colour,updated
0,0,"Frame 332, TL [1285.501953125,362.198760986328...","[1285, 362, 1521, 544]",43045,"[(1403, 451), (1404, 452), (1405, 452), (1404,...","(25, 155, 216)",False
1,1,"Frame 332, TL [560.9329223632812,97.0106353759...","[560, 97, 661, 169]",7246,"[(611, 132), (611, 133), (610, 133), (610, 133...","(250, 46, 20)",False
2,4,"Frame 332, TL [1178.5438232421875,101.05672454...","[1178, 101, 1229, 160]",3042,"[(1203, 130), (1203, 129), (1203, 130), (1203,...","(167, 88, 173)",False
3,3,"Frame 332, TL [585.1447143554688,73.9786987304...","[585, 73, 652, 104]",2086,"[(621, 91), (622, 91), (622, 91), (622, 91), (...","(95, 47, 133)",False
4,6,"Frame 332, TL [843.7902221679688,86.5012969970...","[843, 86, 926, 145]",4874,"[(1041, 403), (1025, 369), (1014, 340), (1002,...","(144, 45, 135)",False
5,6,"Frame 332, TL [926.225830078125,78.47048187255...","[926, 78, 1015, 146]",6137,"[(979, 111), (975, 111), (975, 112), (973, 112...","(95, 62, 67)",False
6,7,"Frame 332, TL [911.7423706054688,93.9007644653...","[911, 93, 943, 122]",904,"[(930, 105), (929, 104), (927, 107)]","(152, 185, 100)",False


In [13]:
tracking_hist[84]

,track,detection,bbox,size,line,colour,updated
0,0,"Frame 336, TL [1285.1729736328125,362.93475341...","[1285, 362, 1521, 544]",42914,"[(1403, 451), (1404, 452), (1405, 452), (1404,...","(25, 155, 216)",False
1,1,"Frame 336, TL [561.6839599609375,97.3791656494...","[561, 97, 660, 169]",7142,"[(611, 132), (611, 133), (610, 133), (610, 133...","(250, 46, 20)",False
2,4,"Frame 336, TL [1178.3714599609375,100.69774627...","[1178, 100, 1229, 160]",3063,"[(1203, 130), (1203, 129), (1203, 130), (1203,...","(167, 88, 173)",False
3,3,"Frame 336, TL [584.6585693359375,73.6403884887...","[584, 73, 654, 105]",2209,"[(621, 91), (622, 91), (622, 91), (622, 91), (...","(95, 47, 133)",False
4,6,"Frame 336, TL [834.861572265625,85.76496887207...","[834, 85, 919, 143]",4855,"[(1041, 403), (1025, 369), (1014, 340), (1002,...","(144, 45, 135)",False
5,6,"Frame 336, TL [923.5205688476562,77.7687377929...","[923, 77, 1016, 146]",6417,"[(979, 111), (975, 111), (975, 112), (973, 112...","(95, 62, 67)",False
6,6,"Frame 336, TL [902.5999755859375,91.2682952880...","[902, 91, 943, 139]",1949,"[(922, 115), (916, 116)]","(0, 5, 220)",False


In [14]:
# for index, row in detections_pd.iterrows():
#     print(row['colour'])
#     image = drawRectangleOnImage(image, row['bbox'], track=row['track'],colour=row['colour'])

In [15]:
# plt.figure(figsize=(20, 12))
# plt.imshow(image)

In [16]:
# #Test......
# test_num = 60
# detections_pd = get_detection_dataframe(all_detections[str(test_num)])
# next_detections_pd = get_detection_dataframe(all_detections[str(test_num+1)])

In [17]:
# detections_pd

In [18]:
# next_detections_pd

In [19]:
# detections_pd = update_track(detections_pd, next_detections_pd, tolerance=0.5)
# next_detections_pd